# Importing and formatting the coordinate data

Import and preview the data

In [1]:
import cv2
import numpy as np
import pandas as pd
import sys

# Import knee coordinates
# coords_file = pd.read_excel("../data/xy coordinates for knee-aging three cycles 250303.xlsx", engine='openpyxl', sheet_name=None) # More updated Excel import
coords_file = pd.read_excel("../data/adjusted xy coordinates for knee-aging 250403.xlsx", engine='openpyxl', sheet_name=None) # More updated Excel import

# Select data set
dataset_index = 1
cf_keys = ['aging-1', 'aging-2', 'aging-3']
coords_sheet = coords_file[cf_keys[dataset_index]] # Set index = {0,1,2} to choose different data set

# Drop columns with no information
coords_sheet.drop(columns=['Unnamed: 0', 'Unnamed: 5'], axis=1, inplace=True)
print(coords_sheet.columns)

# coords_sheet.iloc[120:124]

Index(['Frame Number', 'Points', 'X', 'Y', 'Frame Number.1', 'Points.1', 'X.1',
       'Y.1', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


Select the coordinate data

In [2]:
# Drop rows with no information
framenum_and_coords_is_na = coords_sheet['Frame Number'].isna() & coords_sheet['X'].isna() & coords_sheet['Y'].isna()
coords_1 = coords_sheet[['Frame Number', 'X', 'Y']].loc[~framenum_and_coords_is_na]

# Drop rows with no informatiom
framenum_and_coords_is_na = coords_sheet['Frame Number.1'].isna() & coords_sheet['X.1'].isna() & coords_sheet['Y.1'].isna()
coords_2 = coords_sheet[['Frame Number.1', 'X.1', 'Y.1']].loc[~framenum_and_coords_is_na]

# Get flexion/extension boundary
flx_ext_boundary = int(coords_2.iloc[0]['Frame Number.1'])
print(flx_ext_boundary)

# Concatenate coords columns into a single df
coords_2.rename(columns={'Frame Number.1': 'Frame Number', 'X.1': 'X', 'Y.1': 'Y'}, inplace=True) # Rename columns for proper concatenation
coords = pd.concat([coords_1, coords_2], axis=0)

# Set new index for easier iterations
coords.set_index("Frame Number", inplace=True)
coords = coords.shift(1)
coords.head(12)

# Forward fill empty frame number indixes
coords.index = coords.index.to_series().fillna(method="ffill").astype(int)
coords.tail(12)

102


,X,Y
Frame Number,,
82.0,NaN,NaN
NaN,577.0,617.0
NaN,602.0,719.0
NaN,636.0,595.0
83.0,620.0,740.0
NaN,577.0,617.0
NaN,602.0,719.0
NaN,640.0,597.0
84.0,618.0,749.0


Get unique frames

In [3]:

unique_frames = coords.index.unique()
unique_frames = unique_frames
print(unique_frames)

curr_f_coords = coords.loc[unique_frames[0]]
print(curr_f_coords.to_numpy())

for uniq_f in unique_frames:
    curr_f_coords = coords.loc[uniq_f]

    pt_n = 0 # Set in {0,1,2,3} to select points {1,2,3,4}
    x_or_y = 0 # Set 0 for x, 1 for y
    # print(curr_f_coords.iloc[pt_n][x_or_y])

Float64Index([ 82.0,   nan,  83.0,  84.0,  85.0,  86.0,  87.0,  88.0,  89.0,
               90.0,  91.0,  92.0,  93.0,  94.0,  95.0,  96.0,  97.0,  98.0,
               99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0,
              108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0,
              117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0],
             dtype='float64', name='Frame Number')
[nan nan]


In [7]:
tuple(coords.loc[124])

(649.0, 759.0)

# Get the knee intensity plots

Segment knee into three parts and get sum of pixel intensities

In [39]:
import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..', 'utils'))
if module_path not in sys.path: # Add to sys.path if not already present
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import cv2
import tifffile as tiff
import utils

# Read image stack
aging_knee = tiff.imread("../data/1 aging_00000221.tif") # Imports image stack as np.ndarray (3 dimensions)

# Get number of frames
num_frames, h, w = aging_knee.shape

# Prepend blank frame to align object with 1-based indexing
aging_knee = np.concatenate( (np.zeros((1,h,w)),aging_knee), axis=0)



In [ ]:
# Iterate over all frames
first_frame = 123 #unique_frames[0]
last_frame = 124 #unique_frames[-1]
curr_frame = first_frame

# Initialize brightness list
knee_intensities = np.zeros((3, unique_frames.shape[0]))
knee_intensities_normalized = np.zeros((3, unique_frames.shape[0]))
knee_total_areas = np.zeros((3, unique_frames.shape[0])) # Save total area of each mask 

while curr_frame <= last_frame:


    # Get frame
    frame = aging_knee[curr_frame, :, :].copy() # true copy

    # Pre-processing
    # frame, translation_mx = utils.centroid_stabilization(frame) # Center the frame based on the 

    'Coordinates plotting'

    # Get coordinates to be plotted
    curr_f_coords = coords.loc[curr_frame]
    print(f"curr_f_coords = {curr_f_coords}")

    # Transform coordinates according to centroid stabilization
    # curr_f_coords = np.column_stack([curr_f_coords.to_numpy(), np.ones(curr_f_coords.shape[0])])
    # curr_f_coords = (translation_mx @ curr_f_coords.T).T[:, 0:2]

    # Store coordinates as integer tuples
    point_coords = [tuple(curr_f_coords[pt_n].astype(int)) for pt_n in range(4)]

    print(f"point_coords = {point_coords}")

    # Plot points
    for x, y in point_coords:
        cv2.circle(frame, (x, y), 3, [255, 255, 255], -1)

    # Draw lines between points 0-1 and 2-3
    cv2.line(frame, point_coords[0], point_coords[1], [255, 255, 255], 1)
    cv2.line(frame, point_coords[2], point_coords[3], [255, 255, 255], 1)
#     cv2.circle(frame, point_coords[3], 3, [255, 255, 255], -1)


    # Draw line between left and right region
    midpoints = [
    ((point_coords[0][0] + point_coords[2][0]) // 2, (point_coords[0][1] + point_coords[2][1]) // 2),
    ((point_coords[1][0] + point_coords[3][0]) // 2, (point_coords[1][1] + point_coords[3][1]) // 2)
    ]
    # cv2.line(frame, midpoints[0], midpoints[1], [255,0,0], 1)


    'Three-parts segmentation'

    # Get left/right mask based on the middle line
    left_right_mask = utils.pixels_left_of_line(frame, midpoints[1], midpoints[0])
    # left_right_mask = utils.crop_square_frame(left_right_mask, 350) # For ease of viewing

    # Get middle mask between the left and right lines
    left_mask = utils.pixels_left_of_line(frame, point_coords[1], point_coords[0])
    right_mask = utils.pixels_left_of_line(frame, point_coords[2], point_coords[3])
    # left_mask = utils.crop_square_frame(left_mask, 350) # For ease of viewing
    # right_mask = utils.crop_square_frame(right_mask, 350) # For ease of viewing
    middle_mask = ~left_mask & ~right_mask

    # Get Otsu's mask
    otsu_mask = cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # otsu_mask = utils.crop_square_frame(otsu_mask, 350) # For ease of viewing

    # Get knee masks
    middle_knee_mask = otsu_mask & middle_mask
    left_knee_mask = otsu_mask & left_right_mask & ~middle_mask #& left_mask
    right_knee_mask = otsu_mask & ~left_right_mask & ~middle_mask #& right_mask

    # Crop frame to square region [TODO: do this before the other operations?]
    # frame = utils.crop_square_frame(frame, 350) # For ease of viewing

    # Resize frame for easier viewing
    pixel_scale = 10.1119 # pixels / mm
    # frame, scaling_factor = utils.rescale_frame(frame, pixel_scale, 0)

    'Get sum of pixel intensities in each part'
    
    # Get left/middle/right knee
    left_knee = frame & left_knee_mask
    middle_knee = frame & middle_knee_mask
    right_knee = frame & right_knee_mask

    # Get total number of non-zero pixels
    left_knee_nonzero = np.sum(left_knee_mask)
    middle_knee_nonzero = np.sum(middle_knee_mask)
    right_knee_nonzero = np.sum(right_knee_mask)

    # Get normalized left/middle/right knee pixel intensities
    curr_idx = curr_frame - first_frame
    modify_data = False # --- OPTIONAL: flag to overwrite recorded data --- 
    if modify_data:
        knee_intensities[0, curr_idx] = np.sum(left_knee) # left intensity
        knee_intensities[1, curr_idx] = np.sum(middle_knee) # middle intensity
        knee_intensities[2, curr_idx] = np.sum(right_knee) # right intensity

        knee_intensities_normalized[0, curr_idx] = np.sum(left_knee) / left_knee_nonzero # left intensity
        knee_intensities_normalized[1, curr_idx] = np.sum(middle_knee) / middle_knee_nonzero # middle intensity
        knee_intensities_normalized[2, curr_idx] = np.sum(right_knee) /right_knee_nonzero # right intensity    

        knee_total_areas[0, curr_idx] = np.sum(left_knee_nonzero)
        knee_total_areas[1, curr_idx] = np.sum(middle_knee_nonzero)
        knee_total_areas[2, curr_idx] = np.sum(right_knee_nonzero)


    'Display frames and information'

    # --- For demonstration: overlay the masks on the frame ---
    # frame = np.maximum.reduce([frame, left_knee_mask, middle_knee_mask, right_knee_mask])

    # Show knee_mask, to verify boundaries and such
    knee_mask = np.maximum.reduce([left_knee, middle_knee, right_knee])

    # Write frame number in bottom left corner of knee_mask
    h,w = knee_mask.shape
    pos = (10, h - 10)
    cv2.putText(knee_mask, str(curr_frame), pos, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.7, 
                color = (255,255,255), thickness = 2, lineType = cv2.LINE_AA)
    cv2.imshow(f"knee_mask, frames {first_frame} to {last_frame}", knee_mask) # Display knee_mask

    # Write coordinates in top-left corner 
    # TODO: print the original points and not the transformed coordinates!! 
    pos1 = (int(3*w//4), 15)
    lines = [tuple(coords.loc[curr_frame].iloc[i].astype(int)) for i in range(0,4)]
    cv2.putText(knee_mask, f"l line: {lines[0]}", pos1, 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, 
            color = (255,255,255), thickness = 1, lineType = cv2.LINE_AA)
    pos2 = (int(3*w//4), 30)
    cv2.putText(knee_mask, f"       {lines[1]}", pos2, 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, 
            color = (255,255,255), thickness = 1, lineType = cv2.LINE_AA)
    pos3 = (int(3*w//4), 45)
    cv2.putText(knee_mask, f"r line: {lines[2]}", pos3, 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, 
            color = (255,255,255), thickness = 1, lineType = cv2.LINE_AA)
    pos4 = (int(3*w//4), 60)
    cv2.putText(knee_mask, f"       {lines[3]}", pos4, 
            fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, 
            color = (255,255,255), thickness = 1, lineType = cv2.LINE_AA)


    # Show other masks, for correctness
    check_frame = left_knee_mask & middle_knee_mask # Check for intersections of different masks (should be empty)
    # cv2.imshow("left_knee_mask", left_knee_mask)
    # cv2.imshow("middle_knee_mask", middle_knee_mask)
    # cv2.imshow("right_knee_mask", right_knee_mask)
    # cv2.imshow("check_frame", check_frame)
    # cv2.imshow("whole_mask", np.maximum.reduce([left_knee_mask, middle_knee_mask, right_knee_mask]))

    # Show frame
    cv2.imshow(f"aging knee frames {first_frame} - {last_frame}", frame)

    # Display enlarged frame
    enlarged_frame = frame.copy()
    pixel_scale = 10.1119 # pixels / mm
    enlarged_frame, _ = utils.rescale_frame(frame, pixel_scale, 1.5)
#     cv2.imshow("enlarged frame", enlarged_frame)


    

    'Miscellaneous technical things'

    # TEMPORARY: knee_video frames for Dr. Liang to check correctness
    frame_out = cv2.hconcat([knee_mask, left_knee_mask, middle_knee_mask, right_knee_mask])
    cv2.imshow(f"{cf_keys[dataset_index]} knee (frames {first_frame}-{last_frame})", frame_out)

    # Write temporary frames to file for Dr. Liang
    fname = f"output\\labeled {cf_keys[dataset_index]} knee frames\\{cf_keys[dataset_index]}_{curr_frame:04d}.png"
    # cv2.imwrite(fname, frame_out) # --- WRITE TO FILE ---


    # Increment frame index
    curr_frame = curr_frame +1

    # Press 'q' to quit
    if cv2.waitKey(500) == ord('q'):
        break

    # Optionally loop video
    loop = True
    if loop and curr_frame > last_frame: 
        curr_frame = first_frame

cv2.destroyAllWindows()

In [ ]:
for fn in np.arange(first_frame, last_frame + 1):

    print(f"{fn}: {knee_intensities[0][fn-first_frame]} / {knee_total_areas[0][fn-first_frame]}")
    # print(knee_intensities[0])

Plot intensities (Normalized)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('default')

# Create 3 plots
fig, axes = plt.subplots(1, 3, figsize=(20, 7))  

# Plot all three intensities
x_values = np.arange(unique_frames.shape[0]) + first_frame
axes[0].plot(x_values, knee_intensities_normalized[0, :], label="Sum of pixel intensities (Normalized)", color='r')
axes[1].plot(x_values, knee_intensities_normalized[1, :], label="Sum of pixel intensities (Normalized)", color='g')
axes[2].plot(x_values, knee_intensities_normalized[2, :], label="Sum of pixel intensities (Normalized)", color='b')

# --- Formatting ---
# Set titles
axes[0].set_title(f"Left knee ({cf_keys[dataset_index]}) pixel intensities (normalized)")
axes[1].set_title(f"Middle knee ({cf_keys[dataset_index]}) pixel intensities (normalized)")
axes[2].set_title(f"Right knee ({cf_keys[dataset_index]}) pixel intensities (normalized)")

# Draw flexion/extension line
flx_ext_pt = flx_ext_boundary # aging-1: 62, aging-2: 102, aging-3: 198
flx_ext_pt_adj = flx_ext_pt - first_frame # e.g. for aging-1, flexion ends at frame 61, extension starts at frame 62
axes[0].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
axes[1].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
axes[2].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")

# Show legends
axes[0].legend()
axes[1].legend()
axes[2].legend()

# Save the 3 side-by-side plot
plt.tight_layout()
plt.savefig(f"output\\{cf_keys[dataset_index]}_intensities_side_by_side_normalized.png", dpi=300, bbox_inches="tight")

# Create plot with all three on the same plot 
plt.figure(figsize=(15,8))
plt.plot(x_values, knee_intensities_normalized[0, :], color = 'r', label="Sum of pixel intensities (left knee)")
plt.plot(x_values, knee_intensities_normalized[1, :], color = 'g', label="Sum of pixel intensities (middle knee)")
plt.plot(x_values, knee_intensities_normalized[2, :], color = 'b', label="Sum of pixel intensities (right knee)")
plt.axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
plt.title(f"{cf_keys[dataset_index]} knee intensity plots (normalized)")

plt.legend()
plt.tight_layout()

plt.savefig(f"output\\{cf_keys[dataset_index]}_intensities_combined_normalized.png", dpi=300, bbox_inches="tight")


# Show the plots
plt.show()


Plot intensities (Not Normalized)

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('default')

# Create 3 plots
fig, axes = plt.subplots(1, 3, figsize=(20, 7))  

# Plot all three intensities
x_values = np.arange(unique_frames.shape[0]) + first_frame
axes[0].plot(x_values, knee_intensities[0, :], label="Sum of pixel intensities", color='r')
axes[1].plot(x_values, knee_intensities[1, :], label="Sum of pixel intensities", color='g')
axes[2].plot(x_values, knee_intensities[2, :], label="Sum of pixel intensities", color='b')

# --- Formatting ---
# Set titles
axes[0].set_title(f"Left knee ({cf_keys[dataset_index]}) pixel intensities (not normalized)")
axes[1].set_title(f"Middle knee ({cf_keys[dataset_index]}) pixel intensities (not normalized)")
axes[2].set_title(f"Right knee ({cf_keys[dataset_index]}) pixel intensities (not normalized)")

# Draw flexion/extension line
flx_ext_pt = 102 #aging-1: 62, aging-2: 102, aging-3: 198
flx_ext_pt_adj = flx_ext_pt - first_frame # e.g. for aging-1, flexion ends at frame 61, extension starts at frame 62
axes[0].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
axes[1].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
axes[2].axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")

# Show legends
axes[0].legend()
axes[1].legend()
axes[2].legend()

# Save the 3 side-by-side plot
plt.tight_layout()
plt.savefig(f"output\\{cf_keys[dataset_index]}_intensities_side_by_side.png", dpi=300, bbox_inches="tight")

# Create plot with all three on the same plot 
plt.figure(figsize=(15,8))
plt.plot(x_values, knee_intensities[0, :], color = 'r', label="Sum of pixel intensities (left knee)");
plt.plot(x_values, knee_intensities[1, :], color = 'g', label="Sum of pixel intensities (middle knee)");
plt.plot(x_values, knee_intensities[2, :], color = 'b', label="Sum of pixel intensities (right knee)");
plt.axvline(flx_ext_boundary, color='k', linestyle='--', label=f"Flexion/Extension boundary (frame {flx_ext_boundary})")
plt.title(f"{cf_keys[dataset_index]} knee intensity plots (not normalized)")

plt.legend()
plt.tight_layout()

plt.savefig(f"output\\{cf_keys[dataset_index]}_intensities_combined.png", dpi=300, bbox_inches="tight")

plt.show()


# Meeting Notes (7 Apr 2025)

- **fix knee plotting coordinates** (incorrect indexing? some printed coordinates do not match up)
    - we want to help them feel convinced that collecting more data will be helpful
- generate plots and labeled frames and send to profs 

- calculate difference between $frame_{k+1} - frame_k$ -> **local approximation of rate of change**
    - do this on both the normal and aging knee -> need intensity plots for the normal knee as well 

- for future: add random noise -> simulate new data -> conduct statistical analysis to determine differences between different intensity plots?

# Meeting Notes (1 Apr 2025)
- Dr. Liang wants: 
- ~~sequence of images for each frame with the boundary drawn and lines separating the knee into three parts (only for aging-1 cycle)~~
- ~~(put frame number in the bottom left corner)~~
- **~~get intensity plots for all three cycles~~**
- ~~change the x-axis for the intensity plots to the absolute frame number (not relative frame number)~~



# Meeting Notes (31 Mar 2025)
- ~~make one plot for each cycle (aging-1, aging-2, aging-3)~~
- ~~improve plots by adding formatting things~~
- make plot of the average of each cycle? (maybe we need to ask if they want this, or just make it so they have the option)
- ~~make two versions of the plots: one of them is the current version (sum of pixel intensities), the other is normalized plot of pixel intensities (sum of pixel intensities / number of non-zero pixels)~~
- ~~add dashed v-line to show where the flexion and extension is~~
- after meeting: redo normal knee plot and compare with Juan's method
- ~~send Dr. Liang the aging-1 plot (with titles/labels/legend/v-line)~~

- for future: **work on cutting the old knee into the circle slices**
- consider doing statistical analysis to determine whether there is a meaningful difference between normal and aging knee? (maybe difficult)

# Meeting Notes (17 Mar 2025)
- some frames have parallel lines -> maybe the original coordinates are not very accurate?
- ^ try to alter some coordinates to improve the parallel lines (make note of which coordinates we altered)
- come up with fast results to demonstrate that we can do the analysis they want 
- ~~get plot like Juan's because its clear -> ask him for details of his 3-parts analysis code?~~

- try cutting the circle into N different slices, to also show that we can do the proposed analysis for the CFD model 

# TODO:
- ~~get intensity **inside the boundary only**~~ 
- ~~this can be done by taking the intersection of the masks and the knee mask~~
- ~~send three parts intensity plot to Profs~~ 
- then work on cutting the circle into N slices 

program: "imageJ file" -> "imageJ file (centered)" 